# Explantory Data Analysis Notebook


In [1]:
library(tidyverse)
library(tidymodels)
library(cowplot)
library(ggplot2)
library(broom)


── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.1     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.1
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
── Attaching packages ────────────────────────────────────── tidymodels 1.2.0 ──

✔ broom        1.0.5     ✔ rsample      1.2.1
✔ dials        1.3.0     ✔ tune         1.2.1
✔ infer        1.0.7     ✔ workflows    1.1.4
✔ modeldata    1.4.0     ✔ workflowsets 1.1.0
✔ parsnip      1.2.1     ✔ yardstick    1.3.1
✔ recipes      1.1.0     

── Conflicts ───────────────────────────────────────── tidymodels_conflicts() ──
✖ scales::discard() masks purrr::discard()
✖ dplyr::filt

In [32]:
airpnp <- read.csv("data/listings.csv", header = TRUE)
head(airpnp)

,id,name,host_id,host_name,host_since,host_response_time,host_response_rate,host_is_superhost,neighbourhood_cleansed,neighbourhood_group_cleansed,⋯,bathrooms,bedrooms,beds,price,minimum_nights,availability_365,number_of_reviews,review_scores_rating,license,instant_bookable
,<dbl>,<chr>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,⋯,<dbl>,<int>,<int>,<int>,<int>,<int>,<int>,<dbl>,<chr>,<chr>
1,6.703390e+17,Westwood lovely three bedrooms three bathrooms,4780152,Moon,20/01/13,within a few hours,0.96,f,West Los Angeles,City of Los Angeles,⋯,3,3,3,399,30,365,0,NA,,f
2,3.701449e+07,Spanish style lower duplex near Beverly Hills,278288178,Ida,22/07/19,N/A,N/A,f,Beverlywood,City of Los Angeles,⋯,NA,2,NA,NA,30,0,0,NA,,f
3,1.024835e+18,Charming Beverly Hills Home,513813179,Tiana,08/05/23,within a day,0.6,f,Beverly Hills,Other Cities,⋯,3,3,3,434,30,267,0,NA,,f
4,8.507446e+17,Tianpu's warm room with bathroom,432956623,Dan,22/11/21,a few days or more,0.2,f,Temple City,Other Cities,⋯,1,1,1,49,1,364,1,3.00,,f
5,9.539507e+17,"Santa Monica apt, free parking, steps to the beach",528669205,Farkhat,29/07/23,within an hour,1,t,Santa Monica,Other Cities,⋯,1,0,1,231,5,193,44,4.93,Exempt,t
6,1.082614e+18,Practical Private Room #1398 A,557999481,Christos,22/01/24,within a day,1,f,North Hollywood,City of Los Angeles,⋯,0,4,1,62,30,278,0,NA,,f


In [ ]:
airpnp <- airpnp %>%
    select(-id, -name, -host_id, -host_name, -host_response_time, -license)
airpnp <- airpnp[grepl("^\\d{2}/\\d{2}/\\d{2}$", airpnp$host_since), ]    


In [34]:
colSums(is.na(airpnp))

host_since           host_response_rate 
                           0                            0 
           host_is_superhost       neighbourhood_cleansed 
                           0                            0 
neighbourhood_group_cleansed                     latitude 
                           0                            0 
                   longitude                property_type 
                           0                            0 
                   room_type                 accommodates 
                           0                            0 
                   bathrooms                     bedrooms 
                        8239                         3039 
                        beds                        price 
                        8334                         8237 
              minimum_nights             availability_365 
                           0                            0 
           number_of_reviews         review_scores_rating 
                           0                        12146 
            instant_bookable 
                           0

In [35]:
dim(airpnp)

[1] 45533    19

In [36]:
airpnp <- na.omit(airpnp)
dim(airpnp)

[1] 28045    19

In [37]:
colnames(airpnp)

[1] "host_since"                   "host_response_rate"          
 [3] "host_is_superhost"            "neighbourhood_cleansed"      
 [5] "neighbourhood_group_cleansed" "latitude"                    
 [7] "longitude"                    "property_type"               
 [9] "room_type"                    "accommodates"                
[11] "bathrooms"                    "bedrooms"                    
[13] "beds"                         "price"                       
[15] "minimum_nights"               "availability_365"            
[17] "number_of_reviews"            "review_scores_rating"        
[19] "instant_bookable"

In [38]:
str(airpnp)

'data.frame':	28045 obs. of  19 variables:
 $ host_since                  : chr  "22/11/21" "29/07/23" "14/11/12" "19/04/23" ...
 $ host_response_rate          : chr  "0.2" "1" "1" "1" ...
 $ host_is_superhost           : chr  "f" "t" "f" "t" ...
 $ neighbourhood_cleansed      : chr  "Temple City" "Santa Monica" "Del Rey" "Venice" ...
 $ neighbourhood_group_cleansed: chr  "Other Cities" "Other Cities" "City of Los Angeles" "City of Los Angeles" ...
 $ latitude                    : num  34.1 34 34 34 34.1 ...
 $ longitude                   : num  -118 -118 -118 -118 -118 ...
 $ property_type               : chr  "Private room in home" "Entire rental unit" "Entire home" "Entire home" ...
 $ room_type                   : chr  "Private room" "Entire home/apt" "Entire home/apt" "Entire home/apt" ...
 $ accommodates                : int  2 2 7 6 8 5 4 3 2 6 ...
 $ bathrooms                   : num  1 1 1 3.5 4.5 1 1 1 1 2 ...
 $ bedrooms                    : int  1 0 2 4 4 2 2 1 1 2 ...
 $ b

In [39]:
unique(airpnp$host_is_superhost)

[1] "f" "t" ""

In [40]:
airpnp <- airpnp %>%
    mutate(instant_bookable = ifelse(airpnp$instant_bookable == "t", 1, 0),
           host_is_superhost= ifelse(airpnp$host_is_superhost == "t", 1, 0)
    )

In [43]:
property_type_count <- airpnp %>% count(property_type) %>% arrange(desc(n)) %>% head(10)
property_type_count

,property_type,n
,<chr>,<int>
1,Entire home,8509
2,Entire rental unit,6507
3,Private room in home,3584
4,Entire guesthouse,2180
5,Entire condo,1212
6,Entire guest suite,1071
7,Private room in rental unit,754
8,Entire townhouse,536
9,Entire villa,451


In [45]:
top_10 <- property_type_count$property_type

airpnp <- airpnp[airpnp$property_type %in% top_10, ]
dim(airpnp)

[1] 25209    19

In [48]:
head(airpnp)

,host_since,host_response_rate,host_is_superhost,neighbourhood_cleansed,neighbourhood_group_cleansed,latitude,longitude,property_type,room_type,accommodates,bathrooms,bedrooms,beds,price,minimum_nights,availability_365,number_of_reviews,review_scores_rating,instant_bookable
,<chr>,<chr>,<dbl>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<int>,<dbl>,<int>,<int>,<int>,<int>,<int>,<int>,<dbl>,<dbl>
4,22/11/21,0.2,0,Temple City,Other Cities,34.10933,-118.0731,Private room in home,Private room,2,1.0,1,1,49,1,364,1,3.00,0
5,29/07/23,1,1,Santa Monica,Other Cities,34.01559,-118.4941,Entire rental unit,Entire home/apt,2,1.0,0,1,231,5,193,44,4.93,1
7,14/11/12,1,0,Del Rey,City of Los Angeles,33.99017,-118.4225,Entire home,Entire home/apt,7,1.0,2,5,110,30,117,368,4.61,0
8,19/04/23,1,1,Venice,City of Los Angeles,33.98258,-118.4643,Entire home,Entire home/apt,6,3.5,4,4,1179,2,356,35,4.97,0
9,29/06/21,1,1,Beverly Grove,City of Los Angeles,34.08817,-118.3655,Entire home,Entire home/apt,8,4.5,4,4,1244,3,141,10,5.00,0
12,25/02/19,1,1,West Hollywood,Other Cities,34.09130,-118.3519,Entire townhouse,Entire home/apt,5,1.0,2,3,213,2,244,23,4.74,1


,host_since,host_response_rate,host_is_superhost,neighbourhood_cleansed,neighbourhood_group_cleansed,latitude,longitude,property_type,room_type,accommodates,bathrooms,bedrooms,beds,price,minimum_nights,availability_365,number_of_reviews,review_scores_rating,instant_bookable
,<chr>,<chr>,<dbl>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<int>,<dbl>,<int>,<int>,<int>,<int>,<int>,<int>,<dbl>,<dbl>
5,29/07/23,1,1,Santa Monica,Other Cities,34.01559,-118.4941,Entire rental unit,Entire home/apt,2,1.0,0,1,231,5,193,44,4.93,1
8,19/04/23,1,1,Venice,City of Los Angeles,33.98258,-118.4643,Entire home,Entire home/apt,6,3.5,4,4,1179,2,356,35,4.97,0
12,25/02/19,1,1,West Hollywood,Other Cities,34.09130,-118.3519,Entire townhouse,Entire home/apt,5,1.0,2,3,213,2,244,23,4.74,1
14,30/06/16,1,0,Venice,City of Los Angeles,33.99110,-118.4760,Entire home,Entire home/apt,3,1.0,1,1,298,3,244,59,4.95,0
18,22/02/23,1,0,Beverly Hills,Other Cities,34.07282,-118.3943,Entire rental unit,Entire home/apt,6,2.0,2,3,282,2,313,49,4.61,0
19,11/09/20,1,0,Beverly Hills,Other Cities,34.06482,-118.3950,Entire rental unit,Entire home/apt,2,1.0,1,1,153,2,259,6,4.83,1


In [54]:
unique(airpnp$host_since)

[1] "29/07/23" "19/04/23" "25/02/19" "30/06/16" "22/02/23" "11/09/20"
   [7] "02/03/16" "03/06/13" "26/09/13" "24/04/15" "01/12/23" "27/12/14"
  [13] "03/12/15" "19/05/16" "19/07/11" "20/11/13" "18/08/21" "29/07/11"
  [19] "21/11/23" "29/10/19" "07/12/14" "08/08/19" "16/03/22" "29/10/15"
  [25] "06/11/14" "30/03/22" "23/11/19" "20/12/14" "05/03/16" "07/09/11"
  [31] "24/06/13" "07/04/14" "15/11/20" "28/12/10" "18/09/20" "04/12/16"
  [37] "08/01/20" "24/11/12" "17/02/14" "09/06/17" "24/06/24" "18/09/13"
  [43] "18/11/22" "30/08/18" "06/02/11" "16/06/19" "05/06/22" "31/10/20"
  [49] "02/05/18" "14/06/23" "30/09/13" "06/05/13" "04/03/18" "19/01/15"
  [55] "01/02/18" "02/06/11" "20/03/17" "10/02/16" "07/11/22" "30/12/16"
  [61] "13/10/18" "26/02/13" "11/05/18" "17/03/19" "09/11/14" "02/01/17"
  [67] "12/09/15" "29/10/14" "17/12/15" "04/04/14" "22/03/22" "05/06/19"
  [73] "02/02/21" "21/07/13" "19/01/20" "11/02/19" "17/03/21" "07/10/16"
  [79] "23/04/13" "18/12/19" "13/04/22" "23/10/14" "22/11/19" "15/01/19"
  [85] "07/02/18" "10/09/13" "21/10/17" "22/07/15" "07/03/15" "11/12/20"
  [91] "27/07/12" "04/11/20" "18/10/18" "06/08/13" "02/04/24" "16/05/24"
  [97] "19/12/22" "07/01/16" "31/05/15" "30/11/11" "02/09/16" "30/07/12"
 [103] "06/07/17" "29/07/16" "01/06/15" "05/08/16" "13/07/11" "09/07/14"
 [109] "27/10/16" "06/02/13" "30/06/17" "03/09/22" "30/01/19" "16/12/16"
 [115] "05/02/13" "24/03/15" "26/01/24" "19/01/13" "14/11/12" "05/11/19"
 [121] "26/03/19" "21/05/19" "28/08/16" "18/05/17" "05/10/12" "25/05/24"
 [127] "28/01/13" "21/12/14" "19/09/16" "23/04/23" "17/09/14" "19/06/17"
 [133] "22/04/16" "22/05/22" "10/04/12" "14/12/21" "08/02/15" "14/05/24"
 [139] "03/01/21" "06/06/12" "22/01/20" "12/06/12" "03/07/12" "24/04/23"
 [145] "02/05/24" "16/03/13" "27/10/14" "23/06/16" "21/07/10" "31/08/10"
 [151] "04/03/23" "14/01/21" "30/05/24" "02/03/18" "20/10/19" "11/09/13"
 [157] "18/11/09" "02/11/21" "13/06/21" "02/04/11" "19/10/11" "14/01/14"
 [163] "23/01/24" "28/10/16" "04/04/12" "15/02/24" "16/03/21" "17/11/14"
 [169] "25/06/15" "18/08/12" "14/01/18" "16/10/14" "03/06/21" "18/10/14"
 [175] "26/07/12" "10/08/15" "19/10/16" "04/02/18" "05/10/23" "07/08/09"
 [181] "13/09/19" "02/12/22" "02/02/24" "10/11/20" "03/09/11" "09/01/21"
 [187] "22/01/17" "21/03/16" "28/07/14" "19/04/17" "17/08/21" "28/09/14"
 [193] "09/02/23" "13/12/13" "09/09/21" "12/03/19" "11/05/14" "23/08/22"
 [199] "03/06/24" "19/04/24" "28/03/11" "19/03/16" "06/10/23" "07/07/19"
 [205] "28/05/16" "18/10/12" "22/12/22" "27/07/23" "29/06/24" "30/07/16"
 [211] "19/10/18" "25/12/20" "23/01/17" "28/08/09" "08/12/12" "09/03/18"
 [217] "12/11/13" "06/03/12" "11/01/20" "04/03/12" "05/03/20" "28/12/14"
 [223] "10/06/17" "11/03/16" "03/05/16" "25/04/14" "05/11/16" "15/02/12"
 [229] "27/06/12" "09/06/21" "20/10/20" "11/09/16" "04/04/15" "04/07/13"
 [235] "25/02/16" "28/03/24" "29/07/21" "05/02/12" "31/05/16" "27/05/13"
 [241] "18/09/21" "04/08/15" "01/02/13" "10/04/18" "26/09/19" "11/04/18"
 [247] "11/11/12" "07/05/23" "29/08/18" "07/11/14" "13/07/17" "30/04/16"
 [253] "13/03/22" "28/03/16" "18/07/16" "31/03/10" "02/04/22" "15/01/13"
 [259] "12/04/22" "06/07/12" "26/08/15" "08/06/17" "27/06/13" "01/11/19"
 [265] "21/07/22" "25/10/16" "05/05/18" "16/01/14" "24/01/17" "31/03/21"
 [271] "21/05/16" "20/04/16" "01/09/17" "14/03/16" "23/01/12" "14/02/17"
 [277] "31/10/19" "27/07/22" "08/06/14" "16/10/19" "16/07/15" "18/09/15"
 [283] "20/11/17" "13/06/14" "30/10/12" "20/08/14" "09/06/23" "24/01/24"
 [289] "25/07/13" "30/10/23" "07/11/13" "01/10/14" "05/07/14" "11/08/16"
 [295] "16/05/15" "05/07/18" "20/10/10" "14/10/17" "26/10/16" "13/09/18"
 [301] "01/12/16" "15/08/13" "10/01/19" "27/09/15" "17/08/10" "01/06/24"
 [307] "25/10/11" "13/05/23" "16/04/15" "14/08/14" "07/06/17" "17/07/12"
 [313] "23/05/17" "17/07/23" "31/01/13" "18/06/18" "21/10/13" "24/08/13"
 [319] "11/06/14" "18/07/12" "13/11/18" "20/10/15" "02/06/14" "04/09/13"
 [325] "14/11/22" "24/06/20" "08/11/12" "07/07/16" "09/10/23"

In [60]:
airpnp_filtered <- airpnp[grepl("^\\d{2}/\\d{2}/\\d{2}$", airpnp$host_since), ]
unique(airpnp_filtered$host_since)
dim(airpnp_filtered)

[1] "29/07/23" "19/04/23" "25/02/19" "30/06/16" "22/02/23" "11/09/20"
   [7] "02/03/16" "03/06/13" "26/09/13" "24/04/15" "01/12/23" "27/12/14"
  [13] "03/12/15" "19/05/16" "19/07/11" "20/11/13" "18/08/21" "29/07/11"
  [19] "21/11/23" "29/10/19" "07/12/14" "08/08/19" "16/03/22" "29/10/15"
  [25] "06/11/14" "30/03/22" "23/11/19" "20/12/14" "05/03/16" "07/09/11"
  [31] "24/06/13" "07/04/14" "15/11/20" "28/12/10" "18/09/20" "04/12/16"
  [37] "08/01/20" "24/11/12" "17/02/14" "09/06/17" "24/06/24" "18/09/13"
  [43] "18/11/22" "30/08/18" "06/02/11" "16/06/19" "05/06/22" "31/10/20"
  [49] "02/05/18" "14/06/23" "30/09/13" "06/05/13" "04/03/18" "19/01/15"
  [55] "01/02/18" "02/06/11" "20/03/17" "10/02/16" "07/11/22" "30/12/16"
  [61] "13/10/18" "26/02/13" "11/05/18" "17/03/19" "09/11/14" "02/01/17"
  [67] "12/09/15" "29/10/14" "17/12/15" "04/04/14" "22/03/22" "05/06/19"
  [73] "02/02/21" "21/07/13" "19/01/20" "11/02/19" "17/03/21" "07/10/16"
  [79] "23/04/13" "18/12/19" "13/04/22" "23/10/14" "22/11/19" "15/01/19"
  [85] "07/02/18" "10/09/13" "21/10/17" "22/07/15" "07/03/15" "11/12/20"
  [91] "27/07/12" "04/11/20" "18/10/18" "06/08/13" "02/04/24" "16/05/24"
  [97] "19/12/22" "07/01/16" "31/05/15" "30/11/11" "02/09/16" "30/07/12"
 [103] "06/07/17" "29/07/16" "01/06/15" "05/08/16" "13/07/11" "09/07/14"
 [109] "27/10/16" "06/02/13" "30/06/17" "03/09/22" "30/01/19" "16/12/16"
 [115] "05/02/13" "24/03/15" "26/01/24" "19/01/13" "14/11/12" "05/11/19"
 [121] "26/03/19" "21/05/19" "28/08/16" "18/05/17" "05/10/12" "25/05/24"
 [127] "28/01/13" "21/12/14" "19/09/16" "23/04/23" "17/09/14" "19/06/17"
 [133] "22/04/16" "22/05/22" "10/04/12" "14/12/21" "08/02/15" "14/05/24"
 [139] "03/01/21" "06/06/12" "22/01/20" "12/06/12" "03/07/12" "24/04/23"
 [145] "02/05/24" "16/03/13" "27/10/14" "23/06/16" "21/07/10" "31/08/10"
 [151] "04/03/23" "14/01/21" "30/05/24" "02/03/18" "20/10/19" "11/09/13"
 [157] "18/11/09" "02/11/21" "13/06/21" "02/04/11" "19/10/11" "14/01/14"
 [163] "23/01/24" "28/10/16" "04/04/12" "15/02/24" "16/03/21" "17/11/14"
 [169] "25/06/15" "18/08/12" "14/01/18" "16/10/14" "03/06/21" "18/10/14"
 [175] "26/07/12" "10/08/15" "19/10/16" "04/02/18" "05/10/23" "07/08/09"
 [181] "13/09/19" "02/12/22" "02/02/24" "10/11/20" "03/09/11" "09/01/21"
 [187] "22/01/17" "21/03/16" "28/07/14" "19/04/17" "17/08/21" "28/09/14"
 [193] "09/02/23" "13/12/13" "09/09/21" "12/03/19" "11/05/14" "23/08/22"
 [199] "03/06/24" "19/04/24" "28/03/11" "19/03/16" "06/10/23" "07/07/19"
 [205] "28/05/16" "18/10/12" "22/12/22" "27/07/23" "29/06/24" "30/07/16"
 [211] "19/10/18" "25/12/20" "23/01/17" "28/08/09" "08/12/12" "09/03/18"
 [217] "12/11/13" "06/03/12" "11/01/20" "04/03/12" "05/03/20" "28/12/14"
 [223] "10/06/17" "11/03/16" "03/05/16" "25/04/14" "05/11/16" "15/02/12"
 [229] "27/06/12" "09/06/21" "20/10/20" "11/09/16" "04/04/15" "04/07/13"
 [235] "25/02/16" "28/03/24" "29/07/21" "05/02/12" "31/05/16" "27/05/13"
 [241] "18/09/21" "04/08/15" "01/02/13" "10/04/18" "26/09/19" "11/04/18"
 [247] "11/11/12" "07/05/23" "29/08/18" "07/11/14" "13/07/17" "30/04/16"
 [253] "13/03/22" "28/03/16" "18/07/16" "31/03/10" "02/04/22" "15/01/13"
 [259] "12/04/22" "06/07/12" "26/08/15" "08/06/17" "27/06/13" "01/11/19"
 [265] "21/07/22" "25/10/16" "05/05/18" "16/01/14" "24/01/17" "31/03/21"
 [271] "21/05/16" "20/04/16" "01/09/17" "14/03/16" "23/01/12" "14/02/17"
 [277] "31/10/19" "27/07/22" "08/06/14" "16/10/19" "16/07/15" "18/09/15"
 [283] "20/11/17" "13/06/14" "30/10/12" "20/08/14" "09/06/23" "24/01/24"
 [289] "25/07/13" "30/10/23" "07/11/13" "01/10/14" "05/07/14" "11/08/16"
 [295] "16/05/15" "05/07/18" "20/10/10" "14/10/17" "26/10/16" "13/09/18"
 [301] "01/12/16" "15/08/13" "10/01/19" "27/09/15" "17/08/10" "01/06/24"
 [307] "25/10/11" "13/05/23" "16/04/15" "14/08/14" "07/06/17" "17/07/12"
 [313] "23/05/17" "17/07/23" "31/01/13" "18/06/18" "21/10/13" "24/08/13"
 [319] "11/06/14" "18/07/12" "13/11/18" "20/10/15" "02/06/14" "04/09/13"
 [325] "14/11/22" "24/06/20" "08/11/12" "07/07/16" "09/10/23"

[1] 3202   19

In [61]:
airpnp$host_since <- as.Date(airpnp$host_since, format="%d/%m/%y")
head(airpnp)

,host_since,host_response_rate,host_is_superhost,neighbourhood_cleansed,neighbourhood_group_cleansed,latitude,longitude,property_type,room_type,accommodates,bathrooms,bedrooms,beds,price,minimum_nights,availability_365,number_of_reviews,review_scores_rating,instant_bookable
,<date>,<chr>,<dbl>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<int>,<dbl>,<int>,<int>,<int>,<int>,<int>,<int>,<dbl>,<dbl>
5,2023-07-29,1,1,Santa Monica,Other Cities,34.01559,-118.4941,Entire rental unit,Entire home/apt,2,1.0,0,1,231,5,193,44,4.93,1
8,2023-04-19,1,1,Venice,City of Los Angeles,33.98258,-118.4643,Entire home,Entire home/apt,6,3.5,4,4,1179,2,356,35,4.97,0
12,2019-02-25,1,1,West Hollywood,Other Cities,34.09130,-118.3519,Entire townhouse,Entire home/apt,5,1.0,2,3,213,2,244,23,4.74,1
14,2016-06-30,1,0,Venice,City of Los Angeles,33.99110,-118.4760,Entire home,Entire home/apt,3,1.0,1,1,298,3,244,59,4.95,0
18,2023-02-22,1,0,Beverly Hills,Other Cities,34.07282,-118.3943,Entire rental unit,Entire home/apt,6,2.0,2,3,282,2,313,49,4.61,0
19,2020-09-11,1,0,Beverly Hills,Other Cities,34.06482,-118.3950,Entire rental unit,Entire home/apt,2,1.0,1,1,153,2,259,6,4.83,1


In [64]:
airpnp %>% count(host_since)

host_since,n
<date>,<int>
2008-09-16,2
2008-09-17,1
2009-04-08,1
2009-05-30,1
2009-08-07,1
2009-08-28,1
2009-09-03,1
2009-11-18,1
2009-12-08,1


In [63]:
airpnp %>% tail()

,host_since,host_response_rate,host_is_superhost,neighbourhood_cleansed,neighbourhood_group_cleansed,latitude,longitude,property_type,room_type,accommodates,bathrooms,bedrooms,beds,price,minimum_nights,availability_365,number_of_reviews,review_scores_rating,instant_bookable
,<date>,<chr>,<dbl>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<int>,<dbl>,<int>,<int>,<int>,<int>,<int>,<int>,<dbl>,<dbl>
45326,NA,1,0,Beverly Hills,Other Cities,34.07363,-118.4034,Entire home,Entire home/apt,12,4.5,6,6,3800,1,365,1,5.00,0
45328,NA,0.98,0,Santa Monica,Other Cities,34.01072,-118.4953,Entire rental unit,Entire home/apt,2,1.0,0,1,187,1,145,31,4.68,0
45335,NA,1,1,Santa Monica,Other Cities,34.02267,-118.4564,Entire home,Entire home/apt,3,1.5,2,2,250,31,171,39,4.97,0
45382,NA,1,1,Beverly Hills,Other Cities,34.06299,-118.4085,Entire guesthouse,Entire home/apt,4,1.0,2,3,171,3,123,134,4.91,0
45385,NA,0.96,1,Beverly Hills,Other Cities,34.06799,-118.4150,Entire home,Entire home/apt,4,1.0,1,2,152,1,336,16,4.75,1
45394,NA,0.98,0,West Hollywood,Other Cities,34.09189,-118.3475,Entire rental unit,Entire home/apt,4,2.0,2,3,186,2,340,2,3.00,1
